In [1]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/JaredJose/Roblox-Scraper/main/data/robloxQueryParentPost.csv?token=GHSAT0AAAAAAB6VRXEZDUU5K6C2VIZHTCOSZALOUZA')

In [3]:
df.head()

,url,title,author,date,timestamp,text,subreddit,score,upvote ratio
0,https://www.reddit.com/r/roblox/comments/1070m...,My child loves to play Roblox all day,Mkbrossters,2023-01-09,1.673228e+09,Is there anything I can do a a a parent to mak...,t5_2rnsh,107,0.97
1,https://www.reddit.com/r/roblox/comments/110wc...,do I need to buy robux for this game,edtb,2023-02-13,1.676253e+09,I'm a parent and think spending money on in ga...,t5_2rnsh,7,0.77
2,https://www.reddit.com/r/roblox/comments/116qr...,Escape Obbys?,Hammedic,2023-02-19,1.676848e+09,Why do so many of those “escape obbys” involve...,t5_2rnsh,9,0.81
3,https://www.reddit.com/r/roblox/comments/uzchl...,Roblox parental restrictions are the worst I'v...,Coding_Insomnia,2022-05-28,1.653700e+09,"Hi, I am a proud young father of 2 beautiful l...",t5_2rnsh,144,0.99
4,https://www.reddit.com/r/roblox/comments/jiqm6...,Safety Concern (Please read if you’re a parent),TofuGum,2020-10-27,1.603757e+09,"I’m just a normal Roblox player, no one specia...",t5_2rnsh,2330,0.98


In [4]:
df['text']

0     Is there anything I can do a a a parent to mak...
1     I'm a parent and think spending money on in ga...
2     Why do so many of those “escape obbys” involve...
3     Hi, I am a proud young father of 2 beautiful l...
4     I’m just a normal Roblox player, no one specia...
5     Hello there. I’m a moderate gamer myself but d...
6     There really needs to be a way to set limits o...
7     First of all, lemme clarify, I have no problem...
8     So not only do they not have a "delete account...
9     My friend has been on about how he doesn’t hav...
10    I am really infuriated. Roblox refuses to rese...
11    Ok, so the title says it all. I am sick of see...
12                                                    ?
13    You can hear their voice talk but you can’t sp...
14    I can’t wait because I’ve been playing this fo...
15    I looked at the news the other day to find a n...
16    I just downlaoded ropro/rogold a year ago, add...
17    Have you ever saw your sibling play games 

In [5]:
df['text'].str.count('game').sum()

49

In [6]:
text = " ".join(df['text'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [7]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
#print(bi_grams_list)

In [8]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: make sure with a frequency of 3


In [10]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                 term  rank
996      play roblox   4.0
1163  roblox refuses   3.0
177        buy robux   3.0
773        make sure   3.0
300   delete account   3.0
1141   right erasure   3.0
930     parental pin   3.0


In [11]:
df_ques = df['text'].str.endswith('?')
df_no_na = df.dropna(subset=['text'])
df_true = df_no_na.loc[df_ques]
print(df_true['text'])

5     Hello there. I’m a moderate gamer myself but d...
12                                                    ?
17    Have you ever saw your sibling play games you ...
20    I am a father of a 9 yo daugher who loves adop...
22    I created a profile that was under my email ad...
25    Hey guys, so like most people we are mainly do...
Name: text, dtype: object


In [12]:
df['text_processed'] = df['text'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()

0    is there anything i can do a a a parent to mak...
1    i'm a parent and think spending money on in ga...
2    why do so many of those “escape obbys” involve...
3    hi i am a proud young father of 2 beautiful li...
4    i’m just a normal roblox player no one special...
Name: text_processed, dtype: object

In [13]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['anything', 'parent', 'make', 'enjoyable', 'able', 'spend', 'time', 'thing', 'see', 'could', 'useful', 'buying', 'robux', 'want', 'everyday', 'lol', 'tips', 'advice', 'appreciated']


In [15]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#s.makedirs('./results/ldavis_prepared_10')
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

/home/zhz094/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
